# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.3, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
    !tar xf spark-3.5.1-bin-hadoop3.tgz
    !mv spark-3.5.1-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster

In [5]:
import findspark
findspark.init()

spark_url = 'local'

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark Tutorial')\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

In [11]:
path = 'netflix-rotten-tomatoes-metacritic-imdb.csv'

df = spark.read.option("delimiter", ",").option("header", True).csv(path)

df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Languages: string (nullable = true)
 |-- Series or Movie: string (nullable = true)
 |-- Hidden Gem Score: string (nullable = true)
 |-- Country Availability: string (nullable = true)
 |-- Runtime: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Writer: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- View Rating: string (nullable = true)
 |-- IMDb Score: string (nullable = true)
 |-- Rotten Tomatoes Score: string (nullable = true)
 |-- Metacritic Score: string (nullable = true)
 |-- Awards Received: string (nullable = true)
 |-- Awards Nominated For: string (nullable = true)
 |-- Boxoffice: string (nullable = true)
 |-- Release Date: string (nullable = true)
 |-- Netflix Release Date: string (nullable = true)
 |-- Production House: string (nullable = true)
 |-- Netflix Link: string (nullable = true)
 |-- IMDb Link: string (null

In [12]:
df.show(5)

+-------------------+--------------------+--------------------+----------------+---------------+----------------+--------------------+------------+---------------+--------------------+--------------------+-----------+----------+---------------------+----------------+---------------+--------------------+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+
|              Title|               Genre|                Tags|       Languages|Series or Movie|Hidden Gem Score|Country Availability|     Runtime|       Director|              Writer|              Actors|View Rating|IMDb Score|Rotten Tomatoes Score|Metacritic Score|Awards Received|Awards Nominated For| Boxoffice|Release Date|Netflix Release Date|    Production House|        Netflix Link|           IMDb Link|             Summary|IMDb Votes|               Image|              

## What is the maximum and average of the overall hidden gem score?

In [13]:
from pyspark.sql.functions import max, avg

df.select(max('Hidden Gem Score'), avg('Hidden Gem Score')).show()

+---------------------+---------------------+
|max(Hidden Gem Score)|avg(Hidden Gem Score)|
+---------------------+---------------------+
|                  9.8|    5.937551386501226|
+---------------------+---------------------+



## How many movies that are available in Korea?

In [14]:
from pyspark.sql.functions import col

df.filter(col('Languages').contains('Korean')).count()

735

## Which director has the highest average hidden gem score?

In [15]:
df.groupBy("Director").agg(avg("Hidden Gem Score").alias("avg_score")).orderBy("avg_score", ascending=False).show()

+--------------------+---------+
|            Director|avg_score|
+--------------------+---------+
|         Dorin Marcu|      9.8|
|    Fernando Escovar|      9.6|
|          Rosa Russo|      9.5|
|         Kate Brooks|      9.5|
|Vincent Bal, Kenn...|      9.5|
|    Ignacio Busquier|      9.5|
|Bill Butler, Will...|      9.5|
|     Charles Officer|      9.4|
|           Ryan Sage|      9.3|
|   Frederico Machado|      9.3|
|    Ashish R. Shukla|      9.3|
|         Lisa France|      9.3|
|Jacqui Morris, Da...|      9.3|
|    Jan Philipp Weyl|      9.3|
|      Aundre Johnson|      9.3|
|        R.J. Bentler|      9.3|
|     Rabeah Ghaffari|      9.3|
|          Oh Jin-Koo|      9.3|
|        Shinkyu Choi|      9.3|
|         André Canto|      9.2|
+--------------------+---------+
only showing top 20 rows



## How many genres are there in the dataset?

In [16]:
from pyspark.sql.functions import split, explode
df.select(explode(split("Genre", ", ")).alias("genre")).distinct().count()

28